In [1]:
import random
import pandas as pd
import numpy as np
import sklearn
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import requests
from io import BytesIO
from scipy import stats
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import tensorflow as tf
import math
import cv2


Using TensorFlow backend.


In [2]:
# dataframes - original data
case_path = './data/thyroidCSV/cases.csv'
coord_path = './data/thyroidCSV/coors.csv'
img_path = './data/thyroidCSV/images.csv'

case_df = pd.read_csv(case_path)
coord_df = pd.read_csv(coord_path)
img_df = pd.read_csv(img_path)

In [3]:
# path to all images in a single folder
all_img_path = './data/all_images_by_coord_id/'
img_ids = ['img_'+str(num) for num in range(1, len(coord_df)+1)]

imgs = [Image.open(all_img_path+im_id) for im_id in img_ids]

In [4]:
# open images. convert to array and radd border to make square imgs

small_imgs = [im for im in imgs if im.size == (560, 315)]
imgs = [np.array(im) for im in imgs]
small_imgs = [np.array(im) for im in small_imgs]


resized_imgs = [cv2.copyMakeBorder(im,0,45,0,0,cv2.BORDER_CONSTANT) for im in small_imgs]


In [5]:
imgs = [Image.fromarray(im) for im in imgs]
resized_imgs = [Image.fromarray(im) for im in resized_imgs]

sizes = [im.size for im in resized_imgs]
print (set(sizes))

{(560, 360)}


In [6]:
#convert back to img files

sizes = [im.size for im in resized_imgs]

sizes = [im.size for im in imgs]

In [7]:
for i in imgs:
    if i.size == (560, 360):
        resized_imgs.append(i)
              

In [8]:
resized_imgs = [np.array(im) for im in resized_imgs]

resized_imgs = [cv2.copyMakeBorder(im,0,200,0,0,cv2.BORDER_CONSTANT) for im in resized_imgs]

resized_imgs = [Image.fromarray(im) for im in resized_imgs]

width = 608
height = 608
new_size = (width, height)

for im in resized_imgs:
    im.thumbnail(new_size)  

sizes = [im.size for im in resized_imgs]
print (set(sizes))


{(560, 560)}


In [9]:

# new_size = (width, height)
# for im in imgs:
#     im.thumbnail(new_size)  

len(resized_imgs)

779

In [10]:
# new_imgs = [im.resize(new_size) for im in imgs]
# imgs = new_imgs



In [11]:
# make folder for resized imgs
# add to path to coord_df
i = 0
new_path = []
basepath = 'data/resized_imgs/'
for im in imgs:
    im.save(basepath+img_ids[i], "PNG")
    new_path.append(basepath+img_ids[i])
    i+=1


In [12]:
# create new filename col data
files = img_df['filename'].values
filenames = ['/data/'+ x for x in files]



In [13]:
# new column in img df
img_df['filename'] = filenames


In [14]:
img_df.head(3)

,id,case_id,filename,mark
0,1,1,/data/cimalab/bening/545/1.jpg,"[{""points"": [{""x"": 413, ""y"": 91}, {""x"": 406, ""..."
1,2,1,/data/cimalab/bening/545/2.jpg,"[{""points"": [{""x"": 291, ""y"": 76}, {""x"": 281, ""..."
2,3,5,/data/cimalab/bening/549/1.jpg,"[{""points"": [{""x"": 292, ""y"": 123}, {""x"": 293, ..."


In [15]:
coord_df['img_file'] = [img_df[img_df['id'] == n]['filename'].values[0] for n in coord_df['image_id']]



In [16]:
coord_df['new_path'] = new_path
coord_df.sample(3)



,id,image_id,x,y,w,h,img_file,new_path
634,635,500,229,110,86,81,/data/cimalab/thyroid/373_2.jpg,data/resized_imgs/img_635
86,87,86,261,121,59,79,/data/cimalab/maling/551/3.jpg,data/resized_imgs/img_87
69,70,69,198,37,118,131,/data/cimalab/bening/631/1.jpg,data/resized_imgs/img_70


In [17]:
case_df.head(3)
coord_df.head(10)
#img_df.head(3)


,id,image_id,x,y,w,h,img_file,new_path
0,1,1,385,91,68,98,/data/cimalab/bening/545/1.jpg,data/resized_imgs/img_1
1,2,2,271,72,100,143,/data/cimalab/bening/545/2.jpg,data/resized_imgs/img_2
2,3,3,292,101,79,56,/data/cimalab/bening/549/1.jpg,data/resized_imgs/img_3
3,4,4,258,95,69,53,/data/cimalab/bening/549/2.jpg,data/resized_imgs/img_4
4,5,5,235,71,104,138,/data/cimalab/bening/550/1.jpg,data/resized_imgs/img_5
5,6,6,204,66,160,166,/data/cimalab/bening/550/2.jpg,data/resized_imgs/img_6
6,7,7,366,69,115,79,/data/cimalab/bening/550/3.jpg,data/resized_imgs/img_7
7,8,8,253,109,65,78,/data/cimalab/bening/551/1.jpg,data/resized_imgs/img_8
8,9,9,252,116,61,82,/data/cimalab/bening/551/2.jpg,data/resized_imgs/img_9
9,10,10,261,121,59,79,/data/cimalab/bening/551/3.jpg,data/resized_imgs/img_10


In [18]:

full_df = pd.merge(img_df, coord_df, left_on='id', right_on = 'image_id', how='inner')
full_df[full_df['id_x'] == full_df['image_id']]


full_df = full_df[['case_id', 'image_id', 'x', 'y', 'w', 'h', 'new_path', 'img_file']]


dup_id = full_df[full_df.image_id.duplicated() == True]
dup_id = set(dup_id.image_id.values)

full_df[full_df['image_id']==36]

coord_dict = {}
for row in full_df.iterrows():
    if str(row[1][1]) in coord_dict.keys():
        coord_dict[str(row[1][1])].append(row[1][2:-2].values)
    else: 
        coord_dict[str(row[1][1])] = [row[1][2:-2].values]



In [19]:
full_df = full_df[full_df.img_file.duplicated() == False]

full_df.image_id = [str(i) for i in full_df.image_id.values]


coords = coord_dict.values()
id_keys = coord_dict.keys()

coord_list_df = pd.DataFrame(list(zip(id_keys, coords)), columns=['image_id', 'coords'], )

coord_list_df.sample(3)

full_df.sample(3)

,case_id,image_id,x,y,w,h,new_path,img_file
402,246,318,130,27,312,164,data/resized_imgs/img_405,/data/cimalab/thyroid/233_2.jpg
653,414,521,235,86,144,111,data/resized_imgs/img_656,/data/cimalab/thyroid/391_2.jpg
463,279,353,130,67,107,90,data/resized_imgs/img_466,/data/cimalab/thyroid/263_1.jpg


In [20]:
full_df = full_df.set_index('image_id').join(coord_list_df.set_index('image_id'))

In [21]:
full_df.sample(100)

,case_id,x,y,w,h,new_path,img_file,coords
image_id,,,,,,,,
481,382,222,93,123,101,data/resized_imgs/img_616,/data/cimalab/thyroid/360_2.jpg,"[[222, 93, 123, 101]]"
287,222,251,102,44,27,data/resized_imgs/img_359,/data/cimalab/thyroid/210_2.jpg,"[[251, 102, 44, 27]]"
429,343,270,103,79,63,data/resized_imgs/img_560,/data/cimalab/thyroid/320_1.jpg,"[[270, 103, 79, 63]]"
602,479,124,78,304,223,data/resized_imgs/img_757,/data/cimalab/thyroid/90_2.jpg,"[[124, 78, 304, 223]]"
575,458,137,99,86,53,data/resized_imgs/img_717,/data/cimalab/thyroid/71_1.jpg,"[[137, 99, 86, 53], [402, 96, 44, 51]]"
595,474,97,97,120,85,data/resized_imgs/img_747,/data/cimalab/thyroid/86_1.jpg,"[[97, 97, 120, 85], [353, 84, 97, 84]]"
92,15,189,37,184,146,data/resized_imgs/img_93,/data/cimalab/maling/561/2.jpg,"[[189, 37, 184, 146]]"
406,325,169,47,199,175,data/resized_imgs/img_535,/data/cimalab/thyroid/304_1.jpg,"[[169, 47, 199, 175]]"
338,253,239,51,121,134,data/resized_imgs/img_420,/data/cimalab/thyroid/24_1.jpg,"[[239, 51, 121, 134]]"


In [22]:
full_df = full_df[['case_id', 'img_file', 'new_path', 'coords']]

full_df.reset_index(level='image_id', col_level=1, inplace=True)

In [23]:
full_df.sample(5)

# full_df[full_df['image_id'] == 201]

,image_id,case_id,img_file,new_path,coords
259,270,208,/data/cimalab/thyroid/199_1.jpg,data/resized_imgs/img_336,"[[156, 146, 98, 89], [346, 142, 73, 83]]"
141,144,106,/data/cimalab/thyroid/105_1.jpg,data/resized_imgs/img_151,"[[99, 54, 112, 80], [359, 60, 91, 85]]"
225,234,178,/data/cimalab/thyroid/170_1.jpg,data/resized_imgs/img_283,"[[128, 67, 58, 47], [379, 87, 77, 41]]"
204,210,160,/data/cimalab/thyroid/154_1.jpg,data/resized_imgs/img_251,"[[129, 88, 113, 76], [348, 113, 129, 72]]"
399,414,333,/data/cimalab/thyroid/311_1.jpg,data/resized_imgs/img_544,"[[247, 72, 93, 87]]"


In [24]:
# Create splits 
train = full_df.sample(frac=0.80,replace=False, random_state=99)
remaining = full_df.loc[~full_df.index.isin(train.index), :]


validate = remaining.sample(frac=.5, replace=False, random_state=99)
test = remaining.loc[~remaining.index.isin(validate),:]



In [34]:
train[train['image_id'] == '2']

,image_id,case_id,img_file,new_path,coords
1,2,1,/data/cimalab/bening/545/2.jpg,data/resized_imgs/img_2,"[[271, 72, 100, 143]]"


In [37]:
train.to_csv('train.csv')
test.to_csv('test.csv')
validate.to_csv('validate.csv')

In [26]:
# VOC format
import xml.etree.ElementTree as ET
import xml.dom.minidom

def scale(num):
    # approx scale percentage
    return num

def prep_for_VOC(df, folder):
    
    roots = []
    for row in df.iterrows():
        filename = 'img_{}.png'.format(row[1][0])
        root = ET.Element('annotation')
        fol_elem = ET.SubElement(root, 'folder')
        fol_elem.text = folder
        file_elem = ET.SubElement(root, 'filename')
        file_elem.text = filename
        path_elem = ET.SubElement(root,'path')
        path_elem.text = ('/Users/kathleensullivan/PythonNYCDSA/Machine_Learning/capstone/data/'+folder+'/'+filename)
        src_elem = ET.SubElement(root, 'source')
        db = ET.SubElement(src_elem, 'database')
        db.text=('Unknown')
        sz_elem = ET.SubElement(root, 'size')
        w_elem = ET.SubElement(sz_elem, 'width')
        w_elem.text = str(width)
        h_elem = ET.SubElement(sz_elem, 'height')
        h_elem.text = str(height)
        d_elem = ET.SubElement(sz_elem, 'depth')
        d_elem.text = '3'
        seg_elem = ET.SubElement(root,'segmented')
        seg_elem.text='0'
    
        i = 0
        for _ in row[1][-1]:
            obj_elem = ET.SubElement(root, 'object')
            name_elem = ET.SubElement(obj_elem,'name')
            name_elem.text = 'lesion'
            pose_elem = ET.SubElement(obj_elem,'pose')
            pose_elem.text = 'Unspecified'
            trunc_elem = ET.SubElement(obj_elem,'truncated')
            trunc_elem.text = '0'
            dif_elem = ET.SubElement(obj_elem,'difficult')
            dif_elem.text = '0'
            
            x, y, w, h = row[1][-1][i]
            bb_elem = ET.SubElement(obj_elem,'bndbox')
            xmin_elem = ET.SubElement(bb_elem,'xmin')
            xmin_elem.text = str(x)
            xmax_elem = ET.SubElement(bb_elem,'xmax')
            xmax_elem.text = str(x + w)
            ymin_elem = ET.SubElement(bb_elem,'ymin')
            ymin_elem.text = str(y)
            ymax_elem = ET.SubElement(bb_elem,'ymax')
            ymax_elem.text = str(y + h)
            i += 1
    
        roots.append(root)
    return roots




In [27]:
from xml.etree.ElementTree import Element, SubElement, Comment, tostring
from xml.dom import minidom


def write_files(df_name):
    str_name =[x for x in globals() if globals()[x] is df_name][0]
    txt_files = []
    num_list = df_name.image_id.values
    trees = prep_for_VOC(df_name, str_name)
    i = 0
    for tree in trees:
        num = num_list[i]
        df = df_name[df_name.image_id == num]
        tree = ET.tostring(tree)
        xmlstr = minidom.parseString(tree)
        xmlstr = xmlstr.toprettyxml() 
        xmlstr = xmlstr[22:].strip()
        with open('./data/{}_annot/img_{}.xml'.format(str_name, num), 'w') as f:
             f.write(xmlstr)
        txt_files.append('./data/{}/img_{}.xml'.format(str_name, num))
        img = Image.open(df.new_path.values[0]).convert('LA')

        img.save('./data/'+str_name+'/img_'+str(num)+'.png', 'PNG')
        i+=1
    f = open('./data/{}.txt'.format(str_name), 'w+')
    for t in txt_files:
        f.write(t+'\n')
    f.close()
    

In [28]:
### create sample tables for quick tests

# train=train.copy(deep=True)
# train = train.sample(30)
# validate = validate.sample(10)
# test = test.sample(10)


In [29]:
#### ERASES EXISTING FILES, write full train, test, validate files

import os, re, os.path
train_dir = 'train'
t_annot_dir = 'train_annot'
valid_dir = 'validate'
v_annot_dir = 'validate_annot'
test_dir = 'test'
test_annot_dir ='test_annot'

dir_list = [train_dir, t_annot_dir, valid_dir, v_annot_dir, test_dir, test_annot_dir]

for d in dir_list:
    for root, dirs, file in os.walk(d):
        os.remove(os.path.join(root, file))


write_files(validate)
write_files(train)
write_files(test)


In [30]:
# Get bboxes


example_res = [{'label': 'lesion',
      'confidence': 0.82214236,
      'topleft': {'x': 111, 'y': 68},
      'bottomright': {'x': 217, 'y': 101}},
     {'label': 'lesion',
      'confidence': 0.6979475,
      'topleft': {'x': 315, 'y': 62},
      'bottomright': {'x': 414, 'y': 106}}]



# for each result
def get_box(box_list, box_index):
    current = box_list[box_index]
    box = current['topleft']['x'], current['topleft']['y'], current['bottomright']['x'], current['bottomright']['y']
    return box

def get_true_box(df, img_id, box_index):
    row = df[df['image_id'] == img_id]
    coor_list = row['coords'].values
    box_info = coor_list[box_index]
    box = box_info[box_index][0], box_info[box_index][1], box_info[box_index][0]+box_info[0][2], box_info[box_index][1] + box_info[box_index][3]
    return box
    




In [31]:
# # iou calc, takes topleft, bottom right for each box

def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    # compute the area of both the prediction and true_box
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

In [ ]:
import cv2
from darkflow.net.build import TFNet
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'

In [35]:
test_paths = ''


# img = cv2.imread('../data/test/img_612.png', cv2.IMREAD_COLOR)
















# example_res = [{'label': 'lesion',
#       'confidence': 0.82214236,
#       'topleft': {'x': 111, 'y': 68},
#       'bottomright': {'x': 217, 'y': 101}},
#      {'label': 'lesion',
#       'confidence': 0.6979475,
#       'topleft': {'x': 315, 'y': 62},
#       'bottomright': {'x': 414, 'y': 106}}]



# box1 = get_box(example_res, 0)
# box2 = get_true_box(train, '10', 0)



,image_id,case_id,img_file,new_path,coords
391,406,325,/data/cimalab/thyroid/304_1.jpg,data/resized_imgs/img_535,"[[169, 47, 199, 175]]"
595,613,485,/data/cimalab/thyroid/96_2.jpg,data/resized_imgs/img_771,"[[215, 81, 199, 165]]"
410,425,340,/data/cimalab/thyroid/318_2.jpg,data/resized_imgs/img_555,"[[258, 95, 69, 53]]"
457,472,376,/data/cimalab/thyroid/355_1.jpg,data/resized_imgs/img_605,"[[266, 70, 137, 121]]"
363,378,303,/data/cimalab/thyroid/285_1.jpg,data/resized_imgs/img_505,"[[286, 97, 151, 143]]"
450,465,371,/data/cimalab/thyroid/350_2.jpg,data/resized_imgs/img_597,"[[189, 37, 184, 146]]"
330,345,271,/data/cimalab/thyroid/256_1.jpg,data/resized_imgs/img_452,"[[322, 73, 123, 109], [89, 69, 181, 124]]"
229,238,181,/data/cimalab/thyroid/173_2.jpg,data/resized_imgs/img_288,"[[214, 50, 161, 155]]"
35,37,26,/data/cimalab/bening/572/1.jpg,data/resized_imgs/img_37,"[[181, 64, 177, 171]]"
197,203,153,/data/cimalab/thyroid/148_1.jpg,data/resized_imgs/img_242,"[[234, 115, 70, 50]]"
